In [27]:
from os.path import dirname, join, abspath
from pyrep import PyRep
from pyrep.robots.arms.panda import Panda
import time
import numpy as np

# Panda

In [41]:
SCENE_FILE = "examples/scene_panda_reach_target.ttt"

pr = PyRep()
pr.launch(SCENE_FILE, headless=False)
pr.start()
agent = Panda()

In [46]:
starting_joint_positions = agent.get_joint_positions()
default_pos, default_quat = agent.get_tip().get_position(), agent.get_tip().get_quaternion()

def move(new_pos):
    new_joint_angles = agent.solve_ik_via_jacobian(new_pos, quaternion=quat)
    agent.set_joint_target_positions(new_joint_angles)
    pr.step()

In [43]:
default_pos

array([1.07810307, 0.02294977, 1.47322607])

In [52]:
for i in range(10):
    move(default_pos)

In [54]:

# while True:
time.sleep(5)
MAX_DELTA = 0.01
STEPS = 10

for i in range(STEPS):
    delta
    move(default_pos - i*np.array([0, 0, DELTA]))

for i in range(STEPS):
#     pass
#     move(default_pos)
    move(default_pos - (STEPS - i - 1) *np.array([0, 0, DELTA]))


In [ ]:


pr.stop()
pr.shutdown()


# Panda reach target


In [5]:
%%time
from os.path import dirname, join, abspath
from pyrep import PyRep
from pyrep.robots.arms.panda import Panda
from pyrep.objects.shape import Shape
from pyrep.const import PrimitiveShape
from pyrep.errors import ConfigurationPathError
import numpy as np
import math

LOOPS = 10
SCENE_FILE = 'examples/scene_panda_reach_target.ttt'
pr = PyRep()
pr.launch(SCENE_FILE, headless=True)
pr.start()
agent = Panda()

# We could have made this target in the scene, but lets create one dynamically
target = Shape.create(type=PrimitiveShape.SPHERE,
                      size=[0.05, 0.05, 0.05],
                      color=[1.0, 0.1, 0.1],
                      static=True, respondable=False)

position_min, position_max = [0.8, -0.2, 1.0], [1.0, 0.2, 1.4]

starting_joint_positions = agent.get_joint_positions()

for i in range(LOOPS):

    # Reset the arm at the start of each 'episode'
    agent.set_joint_positions(starting_joint_positions)

    # Get a random position within a cuboid and set the target position
    pos = list(np.random.uniform(position_min, position_max))
    target.set_position(pos)

    # Get a path to the target (rotate so z points down)
    try:
        path = agent.get_path(
            position=pos, euler=[0, math.radians(180), 0])
    except ConfigurationPathError as e:
        print('Could not find path')
        continue

    # Step the simulation and advance the agent along the path
    done = False
    while not done:
        done = path.step()
        pr.step()

    print('Reached target %d!' % i)

pr.stop()
pr.shutdown()

Reached target 0!
Reached target 1!
Reached target 2!
Reached target 3!
Reached target 4!
Reached target 5!
Reached target 6!
Reached target 7!
Reached target 8!
Reached target 9!
CPU times: user 3.25 s, sys: 80.7 ms, total: 3.34 s
Wall time: 5.54 s


# RL Example

In [3]:
from os.path import dirname, join, abspath
from pyrep import PyRep
from pyrep.robots.arms.panda import Panda
from pyrep.objects.shape import Shape
import numpy as np

SCENE_FILE = 'examples/scene_reinforcement_learning_env.ttt'
POS_MIN, POS_MAX = [0.8, -0.2, 1.0], [1.0, 0.2, 1.4]
EPISODES = 5
EPISODE_LENGTH = 200


class ReacherEnv(object):

    def __init__(self):
        self.pr = PyRep()
        self.pr.launch(SCENE_FILE, headless=False)
        self.pr.start()
        self.agent = Panda()
        self.agent.set_control_loop_enabled(False)
        self.agent.set_motor_locked_at_zero_velocity(True)
        self.target = Shape('target')
        self.agent_ee_tip = self.agent.get_tip()
        self.initial_joint_positions = self.agent.get_joint_positions()

    def _get_state(self):
        # Return state containing arm joint angles/velocities & target position
        return np.concatenate([self.agent.get_joint_positions(),
                               self.agent.get_joint_velocities(),
                               self.target.get_position()])

    def reset(self):
        # Get a random position within a cuboid and set the target position
        pos = list(np.random.uniform(POS_MIN, POS_MAX))
        self.target.set_position(pos)
        self.agent.set_joint_positions(self.initial_joint_positions)
        return self._get_state()

    def step(self, action):
        self.agent.set_joint_target_velocities(action)  # Execute action on arm
        self.pr.step()  # Step the physics simulation
        ax, ay, az = self.agent_ee_tip.get_position()
        tx, ty, tz = self.target.get_position()
        # Reward is negative distance to target
        reward = -np.sqrt((ax - tx) ** 2 + (ay - ty) ** 2 + (az - tz) ** 2)
        return reward, self._get_state()

    def shutdown(self):
        self.pr.stop()
        self.pr.shutdown()


class Agent(object):

    def act(self, state):
        del state
        return list(np.random.uniform(-1.0, 1.0, size=(7,)))

    def learn(self, replay_buffer):
        del replay_buffer
        pass


env = ReacherEnv()
agent = Agent()
replay_buffer = []

for e in range(EPISODES):

    print('Starting episode %d' % e)
    state = env.reset()
    for i in range(EPISODE_LENGTH):
        action = agent.act(state)
        reward, next_state = env.step(action)
        replay_buffer.append((state, action, reward, next_state))
        state = next_state
        agent.learn(replay_buffer)

print('Done!')
env.shutdown()

Starting episode 0
Starting episode 1
Starting episode 2
Starting episode 3
Starting episode 4
Done!
